In [1]:
import pandas as pd
# from tqdm import tqdm
from tqdm.auto import tqdm

tqdm.pandas()

/root/nielsen/churn/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
abonnementsprodukter__historikk_df = pd.read_csv('received_csvs/abonnementsprodukter__historikk.csv')


df_part1 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part1.csv")
df_part2 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part2.csv")
df_part3 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part3.csv")
df_part4 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part4.csv")
df_part5 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part5.csv")
df_part6 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part6.csv")
eierverifiseringer__historikk_df = pd.concat([df_part1, df_part2, df_part3, df_part4, df_part5, df_part6], ignore_index=True)

produktuttak_df = pd.read_csv('received_csvs/produktuttak.csv')
stg_dynamics__contacts_df = pd.read_csv('received_csvs/stg_dynamics__contacts.csv')
nps_svar_df = pd.read_csv('received_csvs/nps_svar.csv').drop(columns="contact_rk.1")

/tmp/ipykernel_869563/2519247929.py:6: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_part3 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part3.csv")
/tmp/ipykernel_869563/2519247929.py:8: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_part5 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part5.csv")


In [3]:
abonnementsprodukter__historikk_df.rename(columns={'subscriptionitem__contact_rk':'contact_rk'}, inplace=True)

In [4]:
len(eierverifiseringer__historikk_df)

10444260

### Only store rows with valid membership

In [5]:
eierverifiseringer__historikk_df = eierverifiseringer__historikk_df[eierverifiseringer__historikk_df['contact_rk'].isin(abonnementsprodukter__historikk_df['contact_rk'])]
produktuttak_df = produktuttak_df[produktuttak_df['contact_rk'].isin(abonnementsprodukter__historikk_df['contact_rk'])]
stg_dynamics__contacts_df = stg_dynamics__contacts_df[stg_dynamics__contacts_df['contact_rk'].isin(abonnementsprodukter__historikk_df['contact_rk'])]
nps_svar_df = nps_svar_df[nps_svar_df['contact_rk'].isin(abonnementsprodukter__historikk_df['contact_rk'])]

In [6]:
len(eierverifiseringer__historikk_df)

7684725

# Cleanup

In [8]:
# Count NaNs in specific columns
nan_counts = (
    abonnementsprodukter__historikk_df[["innmeldingsdato", "startdato", "sluttdato"]]
    .isna()
    .sum()
)

print("Number of NaNs:")
print(nan_counts)

Number of NaNs:
innmeldingsdato    102811
startdato               0
sluttdato          524027
dtype: int64


In [9]:
# Find duplicated contact_rk values
duplicates = (
    stg_dynamics__contacts_df["contact_rk"]
    .value_counts()
    .loc[lambda x: x > 1]
)

# Display how many contact_rk are duplicated
print(f"Number of non-unique contact_rk: {duplicates.shape[0]}")

# If you want to see the list:
print(duplicates)

Number of non-unique contact_rk: 0
Series([], Name: count, dtype: int64)


In [ ]:
# abonnementsprodukter__historikk_df = abonnementsprodukter__historikk_df.dropna(subset=['innmeldingsdato', 'sluttdato'])
abonnementsprodukter__historikk_df = abonnementsprodukter__historikk_df.dropna(subset=['sluttdato'])
nan_count = stg_dynamics__contacts_df[['age']].isna().sum()
stg_dynamics__contacts_df = stg_dynamics__contacts_df.dropna(subset=['age'])

print("Rows with age NaN dropped:\n", nan_count)

age    58195
dtype: int64


# Prep

### Only create a year row if, for that year, the sluttdato is at least 3 weeks (21 days) past the anniversary of the startdato.

In [90]:
TODAY = pd.Timestamp("2025-01-01")

# 1. Parse and clean dates
sub = abonnementsprodukter__historikk_df.copy()

sub["startdato"] = pd.to_datetime(sub["startdato"], errors="coerce")
sub["sluttdato"] = pd.to_datetime(sub["sluttdato"], errors="coerce").fillna(TODAY)

number_dropped = sub[["startdato", "sluttdato"]].isna().sum()
sub = sub.dropna(subset=["startdato", "sluttdato"])

# 2. Filter unreasonable dates
sub = sub[
    (sub["startdato"].dt.year >= 1970)
    & (sub["sluttdato"].dt.year >= 1970)
    & (sub["sluttdato"] <= TODAY)
]

# 3. GAP DETECTION: Split contacts based on gap > 1y + 3w
GAP_THRESHOLD = pd.Timedelta(days=365 + 21)

sub = sub.sort_values(["contact_rk", "startdato"])
sub["prev_sluttdato"] = sub.groupby("contact_rk")["sluttdato"].shift()
sub["gap"] = sub["startdato"] - sub["prev_sluttdato"]

sub["new_membership"] = (
    (sub["gap"] > GAP_THRESHOLD) | (sub["prev_sluttdato"].isna())
).astype(int)

sub["membership_number"] = sub.groupby("contact_rk")["new_membership"].cumsum()

sub["contact_rk_gap"] = sub["contact_rk"] + "-" + sub["membership_number"].astype(str)

sub = sub.drop(columns=["prev_sluttdato", "gap", "new_membership", "membership_number"])


# 4. Generate year_list (only if sluttdato > anniversary + 3 weeks)
def generate_year_list(row):
    years = []
    for year in range(row.startdato.year, row.sluttdato.year + 1):
        month = row.startdato.month
        day = row.startdato.day
        if month == 2 and day == 29:
            if not (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)):
                day = 28  # Handle non-leap years
        anniversary = pd.Timestamp(year=year, month=month, day=day)
        if row.sluttdato >= anniversary + pd.Timedelta(days=21):
            years.append(year)
    return years


sub["year_list"] = sub.progress_apply(generate_year_list, axis=1)

# 5. Explode into membership years
membership_years = (
    sub.explode("year_list", ignore_index=True)
    .rename(columns={"year_list": "year"})
    .dropna(subset=["year"])
    .groupby(
        [
            "contact_rk_gap",  # ← keep your base ID
            "contact_rk",  # ← the suffixed membership ID
            "year",
        ],
        as_index=False,
    )
    .agg(is_household=("er_husstandsmedlem", "max"))
)


print("Number rows coerced:\n", number_dropped)

100%|██████████| 766002/766002 [00:38<00:00, 19810.95it/s]


Number rows coerced:
 startdato    2
sluttdato    0
dtype: int64


In [ ]:
# Create price table
price_data = [
    ("2025-01-01", 1520),
    ("2024-01-15", 1440),
    ("2023-01-15", 1380),
    ("2021-01-15", 1320),
    ("2020-01-15", 1270),
    ("2019-01-15", 1220),
    ("2018-01-15", 1190),
    ("2017-01-15", 1140),
    ("2016-01-15", 1120),
    ("2014-01-15", 1095),
    ("2013-01-15", 1060),
    ("2012-01-15", 1035),
    ("2011-01-15", 980),
    ("2009-01-15", 930),
]

price_df = pd.DataFrame(price_data, columns=["effective_date", "price"])
price_df["effective_date"] = pd.to_datetime(price_df["effective_date"])

# Remove duplicates, keep the latest price
price_df = price_df.sort_values("effective_date").drop_duplicates(
    subset="effective_date", keep="last"
)

# Create a year column for easier join
price_df["year"] = price_df["effective_date"].dt.year

# Now match the correct price to each year
# For each membership_years['year'], find the most recent price year <= that year

membership_years["year"] = membership_years["year"].astype(int)

# Make sure 'year' is the same dtype in both DataFrames
membership_years["year"] = membership_years["year"].astype("int64")
price_df["year"] = price_df["year"].astype("int64")

# Sort both
membership_years = membership_years.sort_values("year")
price_df = price_df.sort_values("year")

# Merge
membership_years = pd.merge_asof(
    membership_years, price_df[["year", "price"]], on="year", direction="backward"
)

# Replace NaN prices with -1
membership_years["price"] = membership_years["price"].fillna(-1)

# Re-sort by contact_rk and year
membership_years = membership_years.sort_values(["contact_rk_gap", "year"]).reset_index(
    drop=True
)

In [ ]:
veh = eierverifiseringer__historikk_df[
    [
        "contact_rk",
        "vehicle_rk",
        "registert_første_gang_dato",
        "forrige_eu_kontroll_dato",
        "neste_eu_kontroll_dato",
        "vraket_dato",
        "årsmodell",
    ]
].copy()

# robust parsing for every date-column in that table
for col in [
    "vraket_dato",
    "forrige_eu_kontroll_dato",
    "neste_eu_kontroll_dato",
    "registert_første_gang_dato",
]:
    veh[col] = pd.to_datetime(veh[col], format="mixed", errors="coerce")

veh["reg_start"] = veh["registert_første_gang_dato"]  # rename for brevity

# robust parsing for every date-column in that table
for col in [
    "vraket_dato",
    "forrige_eu_kontroll_dato",
    "neste_eu_kontroll_dato",
    "registert_første_gang_dato",
]:

    # Apply string replacement only if the column is not yet datetime
    veh[col] = veh[col].astype(str).str.replace("T", " ", regex=False)
    veh[col] = pd.to_datetime(veh[col], errors="coerce")

veh["vraket_dato"] = pd.to_datetime(veh["vraket_dato"])
veh["reg_start"] = pd.to_datetime(veh["registert_første_gang_dato"])
veh["start_year"] = veh["reg_start"].dt.year.fillna(veh["årsmodell"])
veh["end_year"] = veh["vraket_dato"].dt.year.fillna(TODAY.year)

# Remove rows where start or end year is missing
veh = veh.dropna(subset=["start_year", "end_year"])

# Ensure start_year and end_year are integers
veh["start_year"] = veh["start_year"].astype(int)
veh["end_year"] = veh["end_year"].astype(int)

# Build the year list
veh["year_list"] = veh.progress_apply(
    lambda r: list(range(r.start_year, r.end_year + 1)), axis=1
)


veh["year_list"] = veh.progress_apply(
    lambda r: list(range(int(r.start_year), int(r.end_year) + 1)), axis=1
)
veh_exp = veh.explode("year_list", ignore_index=True).rename(
    columns={"year_list": "year"}
)
veh_exp["vehicle_age"] = veh_exp["year"] - veh_exp["årsmodell"]

vehicle_yearly = veh_exp.groupby(["contact_rk", "year"], as_index=False).agg(
    vehicle_count=("vehicle_rk", "nunique"), vehicle_age_mean=("vehicle_age", "mean")
)

/tmp/ipykernel_869563/2643968144.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  veh[col] = pd.to_datetime(veh[col], errors="coerce")
/tmp/ipykernel_869563/2643968144.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  veh[col] = pd.to_datetime(veh[col], errors="coerce")
100%|██████████| 7682018/7682018 [00:59<00:00, 129551.86it/s]


In [ ]:
pu = produktuttak_df.copy()
pu["opprettet_dato"] = pd.to_datetime(pu["opprettet_dato"])
pu["year"] = pu["opprettet_dato"].dt.year
belop_yearly = pu.groupby(["contact_rk", "year"], as_index=False)["beløp"].sum()

In [ ]:
fb = nps_svar_df.copy()
fb["opprettet_dato"] = pd.to_datetime(fb["opprettet_dato"])
fb["year"] = fb["opprettet_dato"].dt.year
feedback_yearly = fb.groupby(["contact_rk", "year"], as_index=False).agg(
    gave_feedback=("score", "size"), mean_feedback=("score", "mean")
)
feedback_yearly["gave_feedback"] = feedback_yearly["gave_feedback"] > 0

In [ ]:
contacts = stg_dynamics__contacts_df[
    ["contact_rk", "birth_date", "gender", "sentralitetsindex"]
].copy()
contacts["birth_date"] = pd.to_datetime(contacts["birth_date"])

In [ ]:
df = (
    membership_years.merge(vehicle_yearly, on=["contact_rk", "year"], how="left")
    .merge(belop_yearly, on=["contact_rk", "year"], how="left")
    .merge(feedback_yearly, on=["contact_rk", "year"], how="left")
    .merge(contacts, on="contact_rk", how="left")
)

# fill NA for numeric columns that should default to 0
df[["vehicle_count", "vehicle_age_mean", "beløp", "mean_feedback"]] = df[
    ["vehicle_count", "vehicle_age_mean", "beløp", "mean_feedback"]
].fillna(0)
df["gave_feedback"] = df["gave_feedback"].fillna(False)

# on-the-fly age in that calendar year
df["age"] = df["year"] - df["birth_date"].dt.year


# Assign sequential year numbers per member
df["membership_year"] = (
    df.groupby("contact_rk_gap")["year"].rank(method="dense").astype(int)
)

/tmp/ipykernel_869563/18373046.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ].fillna(0)
/tmp/ipykernel_869563/18373046.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["gave_feedback"] = df["gave_feedback"].fillna(False)


In [ ]:
last_active_year = (
    membership_years.groupby("contact_rk_gap")["year"].max().rename("last_year")
)
df = df.merge(last_active_year, on="contact_rk_gap")
df["churn"] = ((df["year"] == df["last_year"]) & (df["last_year"] < TODAY.year)).astype(
    int
)
df = df.drop(columns=["last_year", "contact_rk_clean"])

In [ ]:
ordered_cols = [
    "contact_rk_gap",
    "membership_year",
    "year",
    "churn",
    "price",
    "is_household",
    "vehicle_count",
    "vehicle_age_mean",
    "beløp",
    "gave_feedback",
    "mean_feedback",
    "age",
    "gender",
    "sentralitetsindex",
]
df = df[ordered_cols].sort_values(["contact_rk_gap", "year"]).reset_index(drop=True)

In [113]:
# Print the number of rows before removing NaNs
print("Rows before removing NaNs:", len(df))

# Remove all rows with any NaN values
df = df.dropna()

# Print the number of rows after removing NaNs
print("Rows after removing NaNs:", len(df))

Rows before removing NaNs: 2461168
Rows after removing NaNs: 2397032


In [115]:
import uuid

uid = str(uuid.uuid4())

df.to_parquet(f'df-{uid}.parquet.gzip', compression='gzip')